# Revenue Forecasting ML Pipeline - Optimized
## Ridge, Lasso & ElasticNet with Advanced Feature Engineering

### Pipeline Order (Strict):
1. **Feature Engineering** → Create domain-specific features
2. **Leakage Checks** → Detect and remove target leakage
3. **Missing Value Handling** → Fill NaN and inf values
4. **Encoding & Scaling** → StandardScaler for regularization
5. **Correlation & Redundancy Removal** → VIF and threshold filtering
6. **Feature Selection** → RFE, SelectFromModel, VarianceThreshold
7. **Proper Data Splitting** → TimeSeriesSplit for CV
8. **Model Test** → Ridge, Lasso, ElasticNet only

### Key Domain Features (From Business Logic):
- **weighted_revenue**: Expected value = forecast × probability
- **revenue_momentum_1m**: Tracks if forecast is growing
- **loading_factor**: Historical month-3 loading within quarter
- **gap_ratio**: Pipeline maturity (weighted/unweighted)
- **coverage_pipeline**: Pipeline sufficiency to hit target
- **maturity_rate**: Conversion efficiency (POC → Committed)
- **structural_feature**: Quarter loading structure
- **pipeline_stability**: Inverse variance of pipeline (stability score)

---
## STEP 1: Import Libraries

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

# Preprocessing
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import (
    TimeSeriesSplit, cross_val_score, GridSearchCV, 
    RandomizedSearchCV, cross_validate
)

# Feature Selection
from sklearn.feature_selection import (
    SelectKBest, f_regression, mutual_info_regression,
    RFE, RFECV, VarianceThreshold, SelectFromModel
)

# Models - Only Ridge, Lasso, ElasticNet
from sklearn.linear_model import (
    Ridge, Lasso, ElasticNet, 
    RidgeCV, LassoCV, ElasticNetCV
)

# Metrics
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    mean_absolute_percentage_error, median_absolute_error
)

# Statistical tests
from scipy import stats
from scipy.stats import spearmanr, pearsonr

# For VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

print("✓ All libraries imported successfully")

---
## STEP 2: Load and Explore Data

In [ ]:
# Load data
df = pd.read_csv('revenue_data.csv')

print("="*70)
print("DATA OVERVIEW")
print("="*70)
print(f"\nShape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nData Types:\n{df.dtypes}")
print(f"\nMissing Values:\n{df.isnull().sum()}")
print(f"\nFirst 5 rows:")
df.head()

In [ ]:
# Statistical summary
print("="*70)
print("STATISTICAL SUMMARY")
print("="*70)
df.describe()

In [ ]:
# Create date index
df_monthly = df.copy()

# Create proper date column
month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
             'Jul': 7, 'Aug': 8, 'Sept': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}

if 'month' in df_monthly.columns:
    df_monthly['month_num_parsed'] = df_monthly['month'].map(month_map)
    if df_monthly['month_num_parsed'].isnull().any():
        df_monthly['month_num_parsed'] = df_monthly['month_num']
else:
    df_monthly['month_num_parsed'] = df_monthly['month_num']

df_monthly['date'] = pd.to_datetime(
    df_monthly['year'].astype(str) + '-' + df_monthly['month_num_parsed'].astype(str) + '-01'
)

df_monthly = df_monthly.sort_values('date').reset_index(drop=True)
print(f"Date range: {df_monthly['date'].min()} to {df_monthly['date'].max()}")
print(f"Total rows: {len(df_monthly)}")

---
## STEP 3: Advanced Feature Engineering

Creating domain-specific features based on business logic:

### Chain of Thought - Feature Reasoning:

1. **weighted_revenue = forecast × probability**
   - Acts as expected value, stabilizes influence of high-value low-probability deals
   - Preserves signal from smaller high-certainty run rate

2. **revenue_momentum_1m = lag_1 - lag_2**
   - Checks if forecast is growing as we approach future date
   - Positive = increasing, Negative = decreasing

3. **loading_factor = avg(month 3 revenue) / avg(quarter revenue)**
   - Quantifies how much revenue typically loads into month 3
   - E.g., 60% means month 3 captures 60% of quarter

4. **gap_ratio = weighted forecast / unweighted forecast**
   - Pipeline maturity indicator
   - Ratio → 1 means most deals are committed stage

5. **coverage_pipeline = unweighted pipeline / forecast target**
   - Checks if pipeline is sufficient to hit target

6. **maturity_rate = committed(t) / pipeline(t-1)**
   - Conversion efficiency from POC to committed

7. **structural_feature = month3_rev / (month1 + month2 + month3)**
   - Quarter loading structure

8. **pipeline_stability = 1 / variance(pipeline, 3 months)**
   - Inverse variance = stability score

In [ ]:
print("="*70)
print("FEATURE ENGINEERING - PHASE 1: BASE TEMPORAL FEATURES")
print("="*70)

# 1. Quarter Features
df_monthly['quarter'] = ((df_monthly['month_num_parsed'] - 1) // 3) + 1
df_monthly['month_in_quarter'] = ((df_monthly['month_num_parsed'] - 1) % 3) + 1
df_monthly['is_quarter_end'] = (df_monthly['month_in_quarter'] == 3).astype(int)
df_monthly['is_q4'] = (df_monthly['quarter'] == 4).astype(int)
df_monthly['is_year_end'] = (df_monthly['month_num_parsed'] >= 10).astype(int)
df_monthly['is_fiscal_year_end'] = (df_monthly['month_num_parsed'] == 12).astype(int)
print("✓ Quarter indicators created")

# 2. Cyclical Features (encode month as sin/cos for continuity)
df_monthly['month_sin'] = np.sin(2 * np.pi * df_monthly['month_num_parsed'] / 12)
df_monthly['month_cos'] = np.cos(2 * np.pi * df_monthly['month_num_parsed'] / 12)
df_monthly['quarter_sin'] = np.sin(2 * np.pi * df_monthly['quarter'] / 4)
df_monthly['quarter_cos'] = np.cos(2 * np.pi * df_monthly['quarter'] / 4)
print("✓ Cyclical features created")

# 3. Year progress
df_monthly['year_progress'] = (df_monthly['month_num_parsed'] - 1) / 11
df_monthly['quarter_progress'] = (df_monthly['month_in_quarter'] - 1) / 2
print("✓ Progress features created")

In [ ]:
print("\n" + "="*70)
print("FEATURE ENGINEERING - PHASE 2: COMMITTED & PIPELINE BASE FEATURES")
print("="*70)

# Total Committed (no leakage - both are known before actual)
df_monthly['total_committed'] = df_monthly['committed_sign_revenue'] + df_monthly['committed_unsig_revenue']
print("✓ Total committed created")

# Total Pipeline (all forecasted revenue)
df_monthly['total_pipeline'] = (
    df_monthly['committed_sign_revenue'] + 
    df_monthly['committed_unsig_revenue'] + 
    df_monthly['wtd_pipeline_revenue']
)
print("✓ Total pipeline created")

In [ ]:
print("\n" + "="*70)
print("FEATURE ENGINEERING - PHASE 3: WEIGHTED REVENUE (EXPECTED VALUE)")
print("="*70)
print("\n💡 Reasoning: weighted_revenue = forecast × probability")
print("   - Acts as stabilizing force")
print("   - Weights down high-value, low-probability deals")
print("   - Preserves signal from smaller high-certainty deals")

# Weighted Revenue = probability-adjusted expected value
# avg_prob_pct is the probability, apply it to pipeline
df_monthly['weighted_pipeline_revenue'] = (
    df_monthly['wtd_pipeline_revenue'] * df_monthly['avg_prob_pct']
)

# Weighted total forecast (committed at 100%, pipeline at probability)
df_monthly['weighted_total_forecast'] = (
    df_monthly['committed_sign_revenue'] * 1.0 +  # 100% certain
    df_monthly['committed_unsig_revenue'] * 0.90 +  # 90% certain
    df_monthly['wtd_pipeline_revenue'] * df_monthly['avg_prob_pct']  # prob weighted
)

# Alternative weighted snapshot (business logic weights)
df_monthly['weighted_snapshot'] = (
    df_monthly['committed_sign_revenue'] * 1.0 +
    df_monthly['committed_unsig_revenue'] * 0.85 +
    df_monthly['wtd_pipeline_revenue'] * 0.45
)

print("✓ Weighted revenue features created")

In [ ]:
print("\n" + "="*70)
print("FEATURE ENGINEERING - PHASE 4: LAG FEATURES")
print("="*70)

# Lag Features - Committed Signed
for lag in [1, 2, 3]:
    df_monthly[f'committed_sign_lag{lag}'] = df_monthly['committed_sign_revenue'].shift(lag)
print("✓ Committed signed lag features (1,2,3) created")

# Lag Features - Committed Unsigned
for lag in [1, 2]:
    df_monthly[f'committed_unsig_lag{lag}'] = df_monthly['committed_unsig_revenue'].shift(lag)
print("✓ Committed unsigned lag features (1,2) created")

# Lag Features - Pipeline
for lag in [1, 2, 3]:
    df_monthly[f'pipeline_lag{lag}'] = df_monthly['wtd_pipeline_revenue'].shift(lag)
print("✓ Pipeline lag features (1,2,3) created")

# Lag Features - Total Committed
for lag in [1, 2, 3]:
    df_monthly[f'total_committed_lag{lag}'] = df_monthly['total_committed'].shift(lag)
print("✓ Total committed lag features (1,2,3) created")

# Lag Features - Total Pipeline
for lag in [1, 2]:
    df_monthly[f'total_pipeline_lag{lag}'] = df_monthly['total_pipeline'].shift(lag)
print("✓ Total pipeline lag features (1,2) created")

# Lag Features - Weighted Forecast
for lag in [1, 2]:
    df_monthly[f'weighted_forecast_lag{lag}'] = df_monthly['weighted_total_forecast'].shift(lag)
print("✓ Weighted forecast lag features (1,2) created")

In [ ]:
print("\n" + "="*70)
print("FEATURE ENGINEERING - PHASE 5: REVENUE MOMENTUM")
print("="*70)
print("\n💡 Reasoning: revenue_momentum_1m = lag_1 - lag_2")
print("   - Positive = forecast revenue is increasing")
print("   - Negative = forecast revenue is decreasing")

# Revenue Momentum (1-month)
df_monthly['committed_sign_momentum_1m'] = (
    df_monthly['committed_sign_lag1'] - df_monthly['committed_sign_lag2']
)
df_monthly['total_committed_momentum_1m'] = (
    df_monthly['total_committed_lag1'] - df_monthly['total_committed_lag2']
)
df_monthly['pipeline_momentum_1m'] = (
    df_monthly['pipeline_lag1'] - df_monthly['pipeline_lag2']
)
df_monthly['weighted_forecast_momentum_1m'] = (
    df_monthly['weighted_forecast_lag1'] - df_monthly['weighted_forecast_lag2']
)
print("✓ 1-month momentum features created")

# Revenue Momentum (2-month)
df_monthly['committed_sign_momentum_2m'] = (
    df_monthly['committed_sign_lag1'] - df_monthly['committed_sign_lag3']
)
df_monthly['total_committed_momentum_2m'] = (
    df_monthly['total_committed_lag1'] - df_monthly['total_committed_lag3']
)
print("✓ 2-month momentum features created")

# Acceleration (second derivative)
df_monthly['committed_sign_acceleration'] = (
    (df_monthly['committed_sign_lag1'] - df_monthly['committed_sign_lag2']) - 
    (df_monthly['committed_sign_lag2'] - df_monthly['committed_sign_lag3'])
)
print("✓ Acceleration features created")

In [ ]:
print("\n" + "="*70)
print("FEATURE ENGINEERING - PHASE 6: ROLLING STATISTICS")
print("="*70)

# Rolling Mean - Committed Signed
for window in [3, 6]:
    df_monthly[f'committed_sign_rolling_mean_{window}'] = \
        df_monthly['committed_sign_revenue'].rolling(window=window, min_periods=1).mean()
print("✓ Committed signed rolling means (3,6) created")

# Rolling Std - Committed Signed
df_monthly['committed_sign_rolling_std_3'] = \
    df_monthly['committed_sign_revenue'].rolling(window=3, min_periods=1).std()
df_monthly['committed_sign_rolling_std_6'] = \
    df_monthly['committed_sign_revenue'].rolling(window=6, min_periods=1).std()
print("✓ Committed signed rolling std (3,6) created")

# Rolling Mean/Std - Pipeline
for window in [3, 6]:
    df_monthly[f'pipeline_rolling_mean_{window}'] = \
        df_monthly['wtd_pipeline_revenue'].rolling(window=window, min_periods=1).mean()
df_monthly['pipeline_rolling_std_3'] = \
    df_monthly['wtd_pipeline_revenue'].rolling(window=3, min_periods=1).std()
print("✓ Pipeline rolling stats created")

# Rolling Mean - Total Committed
for window in [3, 6]:
    df_monthly[f'total_committed_rolling_mean_{window}'] = \
        df_monthly['total_committed'].rolling(window=window, min_periods=1).mean()
print("✓ Total committed rolling means (3,6) created")

# Rolling Min/Max
df_monthly['committed_sign_rolling_min_3'] = \
    df_monthly['committed_sign_revenue'].rolling(window=3, min_periods=1).min()
df_monthly['committed_sign_rolling_max_3'] = \
    df_monthly['committed_sign_revenue'].rolling(window=3, min_periods=1).max()
print("✓ Rolling min/max features created")

In [ ]:
print("\n" + "="*70)
print("FEATURE ENGINEERING - PHASE 7: GAP RATIO (PIPELINE MATURITY)")
print("="*70)
print("\n💡 Reasoning: gap_ratio = weighted forecast / unweighted forecast")
print("   - Ratio approaching 1 = most deals are committed stage")
print("   - Lower ratio = more uncertain pipeline deals")

# Gap Ratio = weighted/unweighted (pipeline maturity indicator)
df_monthly['gap_ratio'] = (
    df_monthly['weighted_total_forecast'] / 
    (df_monthly['total_pipeline'] + 1)  # +1 to avoid division by zero
)

# Simplified gap: committed / total_pipeline
df_monthly['commitment_gap'] = (
    df_monthly['total_committed'] / 
    (df_monthly['total_pipeline'] + 1)
)

print("✓ Gap ratio features created")

In [ ]:
print("\n" + "="*70)
print("FEATURE ENGINEERING - PHASE 8: COVERAGE PIPELINE")
print("="*70)
print("\n💡 Reasoning: coverage = unweighted pipeline / forecast target")
print("   - Checks if pipeline is enough to hit target")
print("   - >1 = sufficient coverage")

# Coverage pipeline ratio
# Using rolling mean as "target" estimate
target_estimate = df_monthly['total_committed_rolling_mean_6']
df_monthly['coverage_pipeline'] = (
    df_monthly['total_pipeline'] / 
    (target_estimate + 1)
)

# Pipeline coverage vs weighted forecast
df_monthly['pipeline_coverage_weighted'] = (
    df_monthly['total_pipeline'] / 
    (df_monthly['weighted_total_forecast'] + 1)
)

print("✓ Coverage pipeline features created")

In [ ]:
print("\n" + "="*70)
print("FEATURE ENGINEERING - PHASE 9: MATURITY RATE")
print("="*70)
print("\n💡 Reasoning: maturity_rate = committed(t) / pipeline(t-1)")
print("   - Conversion efficiency from POC to Committed")
print("   - High rate = good pipeline conversion")

# Maturity Rate = committed this month / pipeline last month
df_monthly['maturity_rate_signed'] = (
    df_monthly['committed_sign_revenue'] / 
    (df_monthly['pipeline_lag1'] + 1)
)

df_monthly['maturity_rate_total'] = (
    df_monthly['total_committed'] / 
    (df_monthly['total_pipeline_lag1'] + 1)
)

# Conversion efficiency - committed growth relative to pipeline
df_monthly['conversion_efficiency'] = (
    (df_monthly['committed_sign_revenue'] - df_monthly['committed_sign_lag1']) / 
    (df_monthly['pipeline_lag1'] + 1)
)

print("✓ Maturity rate features created")

In [ ]:
print("\n" + "="*70)
print("FEATURE ENGINEERING - PHASE 10: LOADING FACTOR (QUARTER DISTRIBUTION)")
print("="*70)
print("\n💡 Reasoning: loading_factor = avg(month 3 revenue) / avg(quarter revenue)")
print("   - Quantifies how much revenue loads into month 3 of quarter")
print("   - E.g., 60% means 60% of quarter revenue comes from month 3")

# Calculate quarterly averages
df_monthly['quarter_key'] = df_monthly['year'].astype(str) + '_Q' + df_monthly['quarter'].astype(str)

# Create rolling quarter average
df_monthly['quarter_rolling_avg'] = df_monthly.groupby('quarter_key')['committed_sign_revenue'].transform('mean')

# Loading factor = current month / quarter average
df_monthly['loading_factor'] = (
    df_monthly['committed_sign_revenue'] / 
    (df_monthly['quarter_rolling_avg'] + 1)
)

# Month-in-quarter loading pattern
df_monthly['month_loading_weight'] = df_monthly['month_in_quarter'] / 3.0

print("✓ Loading factor features created")

In [ ]:
print("\n" + "="*70)
print("FEATURE ENGINEERING - PHASE 11: STRUCTURAL FEATURE")
print("="*70)
print("\n💡 Reasoning: structural_feature = month3_rev / (month1 + month2 + month3)")
print("   - Measures quarter loading structure")
print("   - High value = back-loaded quarter")

# Cumulative within quarter
df_monthly['quarter_cumsum'] = df_monthly.groupby('quarter_key')['committed_sign_revenue'].cumsum()

# Structural feature - proportion of quarter revenue at this point
df_monthly['structural_feature'] = (
    df_monthly['committed_sign_revenue'] / 
    (df_monthly['quarter_cumsum'] + 1)
)

# Quarter completion ratio
quarter_totals = df_monthly.groupby('quarter_key')['committed_sign_revenue'].transform('sum')
df_monthly['quarter_completion_pct'] = df_monthly['quarter_cumsum'] / (quarter_totals + 1)

print("✓ Structural features created")

In [ ]:
print("\n" + "="*70)
print("FEATURE ENGINEERING - PHASE 12: PIPELINE STABILITY SCORE")
print("="*70)
print("\n💡 Reasoning: stability = 1 / variance(pipeline, 3 months)")
print("   - Inverse of variance = stability score")
print("   - High stability = consistent pipeline")

# Pipeline Stability Score = 1 / variance (3 months)
pipeline_variance_3m = df_monthly['wtd_pipeline_revenue'].rolling(window=3, min_periods=1).var()
df_monthly['pipeline_stability_3m'] = 1 / (pipeline_variance_3m + 1e6)  # Add large constant to avoid inf

# Committed stability
committed_variance_3m = df_monthly['committed_sign_revenue'].rolling(window=3, min_periods=1).var()
df_monthly['committed_stability_3m'] = 1 / (committed_variance_3m + 1e6)

# Coefficient of Variation (normalized stability)
df_monthly['pipeline_cv'] = (
    df_monthly['pipeline_rolling_std_3'] / 
    (df_monthly['pipeline_rolling_mean_3'] + 1)
)
df_monthly['committed_sign_cv'] = (
    df_monthly['committed_sign_rolling_std_3'] / 
    (df_monthly['committed_sign_rolling_mean_3'] + 1)
)

print("✓ Pipeline stability features created")

In [ ]:
print("\n" + "="*70)
print("FEATURE ENGINEERING - PHASE 13: RATIO FEATURES")
print("="*70)

# Ratio Features
df_monthly['committed_sign_ratio'] = (
    df_monthly['committed_sign_revenue'] / 
    (df_monthly['total_committed'] + 1)
)
df_monthly['pipeline_to_committed_ratio'] = (
    df_monthly['wtd_pipeline_revenue'] / 
    (df_monthly['total_committed'] + 1)
)
df_monthly['signed_unsigned_ratio'] = (
    df_monthly['committed_sign_revenue'] / 
    (df_monthly['committed_unsig_revenue'] + 1)
)

# Signed to total ratio
df_monthly['signed_to_total_ratio'] = (
    df_monthly['committed_sign_revenue'] / 
    (df_monthly['total_pipeline'] + 1)
)

print("✓ Ratio features created")

In [ ]:
print("\n" + "="*70)
print("FEATURE ENGINEERING - PHASE 14: EMA FEATURES")
print("="*70)

# EMA - Committed Signed
for span in [3, 6]:
    df_monthly[f'committed_sign_ema_{span}'] = \
        df_monthly['committed_sign_revenue'].ewm(span=span, adjust=False).mean()
print("✓ Committed signed EMA (3,6) created")

# EMA - Total Committed
for span in [3, 6]:
    df_monthly[f'total_committed_ema_{span}'] = \
        df_monthly['total_committed'].ewm(span=span, adjust=False).mean()
print("✓ Total committed EMA (3,6) created")

# EMA - Pipeline
df_monthly['pipeline_ema_3'] = df_monthly['wtd_pipeline_revenue'].ewm(span=3, adjust=False).mean()
print("✓ Pipeline EMA created")

In [ ]:
print("\n" + "="*70)
print("FEATURE ENGINEERING - PHASE 15: TREND & DEVIATION FEATURES")
print("="*70)

# Trend Features (deviation from rolling mean)
df_monthly['committed_sign_trend_3'] = (
    df_monthly['committed_sign_revenue'] - df_monthly['committed_sign_rolling_mean_3']
)
df_monthly['committed_sign_trend_6'] = (
    df_monthly['committed_sign_revenue'] - df_monthly['committed_sign_rolling_mean_6']
)
df_monthly['total_committed_trend'] = (
    df_monthly['total_committed'] - df_monthly['total_committed_rolling_mean_3']
)
print("✓ Trend features created")

# Relative strength (current vs rolling)
df_monthly['committed_sign_relative_strength'] = (
    df_monthly['committed_sign_revenue'] / 
    (df_monthly['committed_sign_rolling_mean_3'] + 1)
)
print("✓ Relative strength features created")

# Z-score features
df_monthly['committed_sign_zscore'] = (
    (df_monthly['committed_sign_revenue'] - df_monthly['committed_sign_rolling_mean_6']) / 
    (df_monthly['committed_sign_rolling_std_6'] + 1)
)
print("✓ Z-score features created")

In [ ]:
print("\n" + "="*70)
print("FEATURE ENGINEERING - PHASE 16: YEAR-TO-DATE FEATURES")
print("="*70)

# Year-to-Date Cumulative
df_monthly['ytd_committed_sign'] = df_monthly.groupby('year')['committed_sign_revenue'].cumsum()
df_monthly['ytd_total_committed'] = df_monthly.groupby('year')['total_committed'].cumsum()
df_monthly['ytd_pipeline'] = df_monthly.groupby('year')['wtd_pipeline_revenue'].cumsum()
print("✓ Year-to-date cumulative features created")

# YTD average
df_monthly['ytd_avg_committed'] = (
    df_monthly['ytd_committed_sign'] / df_monthly['month_num_parsed']
)
print("✓ Year-to-date average features created")

In [ ]:
print("\n" + "="*70)
print("FEATURE ENGINEERING - PHASE 17: VELOCITY FEATURES")
print("="*70)

# Commitment velocity
df_monthly['commitment_velocity'] = df_monthly['total_committed'] - df_monthly['total_committed_lag1']
df_monthly['commitment_velocity_3m'] = df_monthly['commitment_velocity'].rolling(window=3, min_periods=1).mean()
print("✓ Commitment velocity features created")

# Pipeline velocity
df_monthly['pipeline_velocity'] = df_monthly['wtd_pipeline_revenue'] - df_monthly['pipeline_lag1']
print("✓ Pipeline velocity features created")

# Pipeline to signed flow
df_monthly['pipeline_to_signed_flow'] = (
    df_monthly['committed_sign_revenue'] - df_monthly['wtd_pipeline_revenue']
)
print("✓ Pipeline to signed flow feature created")

In [ ]:
print("\n" + "="*70)
print("FEATURE ENGINEERING - PHASE 18: PERCENTAGE CHANGE FEATURES")
print("="*70)

# Percentage Changes
df_monthly['committed_sign_pct_change'] = df_monthly['committed_sign_revenue'].pct_change()
df_monthly['total_committed_pct_change'] = df_monthly['total_committed'].pct_change()
df_monthly['pipeline_pct_change'] = df_monthly['wtd_pipeline_revenue'].pct_change()
print("✓ Percentage change features created")

In [ ]:
print("\n" + "="*70)
print("FEATURE ENGINEERING - PHASE 19: INTERACTION FEATURES")
print("="*70)

# Month × Quarter interactions
df_monthly['month_quarter_interaction'] = (
    df_monthly['month_in_quarter'] * df_monthly['is_q4']
)

# Momentum × Stability interactions
df_monthly['momentum_stability_interaction'] = (
    df_monthly['committed_sign_momentum_1m'] * df_monthly['committed_stability_3m']
)

# Gap × Coverage interaction
df_monthly['gap_coverage_interaction'] = (
    df_monthly['gap_ratio'] * df_monthly['coverage_pipeline']
)

print("✓ Interaction features created")

---
## STEP 4: Target Leakage Detection

**CRITICAL:** Check for features that have unrealistically high correlation with target.

In [ ]:
print("="*70)
print("TARGET LEAKAGE DETECTION - PHASE 1: CORRELATION CHECK")
print("="*70)

# Check correlation of features with target
numeric_cols = df_monthly.select_dtypes(include=[np.number]).columns.tolist()
exclude_cols = ['year', 'month_num', 'month_num_parsed', 'actual_revenue', 'date']
feature_cols_temp = [col for col in numeric_cols if col not in exclude_cols]

print("\n📊 Features with |correlation| > 0.95 with target (POTENTIAL LEAKAGE):")
print("-" * 50)

leakage_candidates = []
for col in feature_cols_temp:
    if df_monthly[col].nunique() > 1:  # Skip constant columns
        corr = df_monthly[col].corr(df_monthly['actual_revenue'])
        if abs(corr) > 0.95:
            leakage_candidates.append((col, corr))
            print(f"  ⚠️ {col}: {corr:.4f}")

if not leakage_candidates:
    print("  ✓ No features with correlation > 0.95 found")

print(f"\nTotal potential leakage candidates: {len(leakage_candidates)}")

In [ ]:
print("\n" + "="*70)
print("TARGET LEAKAGE DETECTION - PHASE 2: LINEAR COMBINATION CHECK")
print("="*70)

# Check if sum of components equals target (would be leakage)
component_sum = (
    df_monthly['committed_sign_revenue'] + 
    df_monthly['committed_unsig_revenue'] + 
    df_monthly['wtd_pipeline_revenue']
)

# Compare with actual revenue
diff = abs(df_monthly['actual_revenue'] - component_sum)
mean_diff = diff.mean()
max_diff = diff.max()

print(f"\nSum of components vs actual_revenue:")
print(f"  Mean difference: ${mean_diff:,.2f}")
print(f"  Max difference: ${max_diff:,.2f}")

if mean_diff < 100:
    print("\n⚠️ WARNING: Components nearly perfectly sum to target!")
    print("   Using components directly could cause leakage.")
else:
    print("\n✓ Components do NOT perfectly reconstruct target. Safe to proceed.")

In [ ]:
print("\n" + "="*70)
print("TARGET LEAKAGE DETECTION - PHASE 3: BUSINESS LOGIC REVIEW")
print("="*70)

print("""
📋 BUSINESS LOGIC REVIEW:

SAFE FEATURES (known before actual revenue):
  ✓ committed_sign_revenue - Signed deals known before close
  ✓ committed_unsig_revenue - Unsigned deals known before close  
  ✓ wtd_pipeline_revenue - Pipeline known before close
  ✓ All lag features - Historical data, no leakage
  ✓ All rolling statistics - Based on historical data
  ✓ All ratio features - Derived from safe base features

RESTRICTED FEATURES (use with caution):
  ⚠️ avg_prob_pct - Analyst probability guess
     -> Only used for weighting, not as direct predictor
     -> weighted_pipeline_revenue is the derived feature

REMOVED FEATURES:
  ❌ Any feature perfectly correlated (>0.98) with target
""")

---
## STEP 5: Missing Value Handling

In [ ]:
print("="*70)
print("MISSING VALUE HANDLING")
print("="*70)

# Check missing values before handling
missing_before = df_monthly.isnull().sum()
missing_cols = missing_before[missing_before > 0]

print(f"\nColumns with missing values: {len(missing_cols)}")
if len(missing_cols) > 0:
    print("\nMissing value counts:")
    for col, count in missing_cols.items():
        print(f"  {col}: {count}")

# Fill NaN values with 0 for numerical columns
# (Appropriate for lag features where early rows have no history)
df_monthly = df_monthly.fillna(0)

# Replace infinite values
df_monthly = df_monthly.replace([np.inf, -np.inf], 0)

# Verify
missing_after = df_monthly.isnull().sum().sum()
inf_count = np.isinf(df_monthly.select_dtypes(include=[np.number])).sum().sum()

print(f"\n✓ Missing values after handling: {missing_after}")
print(f"✓ Infinite values after handling: {inf_count}")

---
## STEP 6: Proper Data Splitting

In [ ]:
print("="*70)
print("DATA SPLITTING")
print("="*70)

# Define feature columns
exclude_cols = [
    'year', 'month', 'month_num', 'month_num_parsed', 
    'actual_revenue', 'date', 'quarter_key',
    'avg_prob_pct'  # Exclude raw probability (used only for derived features)
]

feature_cols = [
    col for col in df_monthly.columns 
    if col not in exclude_cols 
    and df_monthly[col].dtype in ['int64', 'float64']
]

print(f"\nTotal features before filtering: {len(feature_cols)}")

# Train-Test Split (temporal)
df_model = df_monthly[df_monthly['actual_revenue'] > 0].copy()
train_data = df_model[df_model['year'].isin([2023, 2024])].copy()
test_data = df_model[df_model['year'] == 2025].copy()

print(f"\nTrain (2023-2024): {len(train_data)} months")
print(f"Test (2025): {len(test_data)} months")

# Prepare X and y
X_train_raw = train_data[feature_cols]
y_train = train_data['actual_revenue']
X_test_raw = test_data[feature_cols]
y_test = test_data['actual_revenue']

print(f"\nX_train shape: {X_train_raw.shape}")
print(f"X_test shape: {X_test_raw.shape}")

---
## STEP 7: Correlation & Redundancy Removal

In [ ]:
print("="*70)
print("CORRELATION & REDUNDANCY REMOVAL - PHASE 1: VARIANCE THRESHOLD")
print("="*70)

# Remove low variance features
var_threshold = VarianceThreshold(threshold=0.01)
var_threshold.fit(X_train_raw)

low_var_mask = var_threshold.get_support()
features_after_var = [f for f, m in zip(feature_cols, low_var_mask) if m]
features_removed_var = [f for f, m in zip(feature_cols, low_var_mask) if not m]

print(f"\nFeatures removed due to low variance: {len(features_removed_var)}")
if features_removed_var:
    for f in features_removed_var:
        print(f"  - {f}")
print(f"\nFeatures remaining: {len(features_after_var)}")

In [ ]:
print("\n" + "="*70)
print("CORRELATION & REDUNDANCY REMOVAL - PHASE 2: MULTICOLLINEARITY")
print("="*70)

def remove_multicollinearity(df, features, threshold=0.85):
    """Remove features with correlation > threshold"""
    corr_matrix = df[features].corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    # Find features to drop (keep first, drop second)
    to_drop = []
    for column in upper.columns:
        if any(upper[column] > threshold):
            # Find the feature it's correlated with
            correlated_with = upper.index[upper[column] > threshold].tolist()
            to_drop.append(column)
    
    return [f for f in features if f not in to_drop], to_drop

features_after_corr, dropped_corr = remove_multicollinearity(
    train_data, features_after_var, threshold=0.85
)

print(f"\nFeatures removed due to multicollinearity (>0.85): {len(dropped_corr)}")
print(f"Features remaining: {len(features_after_corr)}")

In [ ]:
print("\n" + "="*70)
print("CORRELATION & REDUNDANCY REMOVAL - PHASE 3: TARGET CORRELATION")
print("="*70)

# Calculate correlation with target
target_corr = train_data[features_after_corr + ['actual_revenue']].corr()['actual_revenue'].drop('actual_revenue')
target_corr_sorted = target_corr.abs().sort_values(ascending=False)

print("\nTop 25 Features by Correlation with Target:")
for i, (feat, corr) in enumerate(target_corr_sorted.head(25).items(), 1):
    sign = '+' if target_corr[feat] > 0 else '-'
    print(f"  {i:2d}. {feat}: {sign}{abs(corr):.4f}")

In [ ]:
# Update X with filtered features
X_train_filtered = train_data[features_after_corr]
X_test_filtered = test_data[features_after_corr]

print(f"\nFiltered feature count: {len(features_after_corr)}")

---
## STEP 8: Feature Scaling

In [ ]:
print("="*70)
print("FEATURE SCALING")
print("="*70)

# Use StandardScaler (better for regularized models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_filtered)
X_test_scaled = scaler.transform(X_test_filtered)

print(f"✓ Features scaled with StandardScaler")
print(f"\nX_train_scaled shape: {X_train_scaled.shape}")
print(f"X_test_scaled shape: {X_test_scaled.shape}")
print(f"\ny_train range: ${y_train.min():,.0f} - ${y_train.max():,.0f}")
print(f"y_test range: ${y_test.min():,.0f} - ${y_test.max():,.0f}")

---
## STEP 9: Feature Selection (Multiple Methods)

In [ ]:
print("="*70)
print("FEATURE SELECTION - METHOD 1: RFE (Recursive Feature Elimination)")
print("="*70)

# Use Ridge for RFE (stable for small datasets)
ridge_rfe = Ridge(alpha=10.0)

# RFE with optimal number of features
n_features_rfe = min(15, len(features_after_corr))
rfe = RFE(estimator=ridge_rfe, n_features_to_select=n_features_rfe, step=1)
rfe.fit(X_train_scaled, y_train)

# Get selected features
rfe_selected = [f for f, s in zip(features_after_corr, rfe.support_) if s]

print(f"\nRFE selected {len(rfe_selected)} features:")
for i, feat in enumerate(rfe_selected, 1):
    corr = target_corr[feat] if feat in target_corr else 0
    print(f"  {i:2d}. {feat} (corr: {corr:.4f})")

In [ ]:
print("\n" + "="*70)
print("FEATURE SELECTION - METHOD 2: SelectFromModel (Lasso)")
print("="*70)

# Use Lasso for feature selection (inherent L1 sparsity)
lasso_selector = LassoCV(cv=TimeSeriesSplit(n_splits=3), max_iter=10000, random_state=42)
lasso_selector.fit(X_train_scaled, y_train)

sfm = SelectFromModel(lasso_selector, prefit=True)
sfm_mask = sfm.get_support()
sfm_selected = [f for f, m in zip(features_after_corr, sfm_mask) if m]

print(f"\nLasso SelectFromModel selected {len(sfm_selected)} features:")
for i, feat in enumerate(sfm_selected, 1):
    corr = target_corr[feat] if feat in target_corr else 0
    print(f"  {i:2d}. {feat} (corr: {corr:.4f})")

In [ ]:
print("\n" + "="*70)
print("FEATURE SELECTION - METHOD 3: Mutual Information")
print("="*70)

# Compute mutual information scores
mi_scores = mutual_info_regression(X_train_scaled, y_train, random_state=42)
mi_df = pd.DataFrame({
    'feature': features_after_corr,
    'mi_score': mi_scores
}).sort_values('mi_score', ascending=False)

# Select top features by MI
n_features_mi = min(15, len(features_after_corr))
mi_selected = mi_df.head(n_features_mi)['feature'].tolist()

print(f"\nTop {n_features_mi} features by Mutual Information:")
for i, row in mi_df.head(n_features_mi).iterrows():
    print(f"  {row['feature']}: {row['mi_score']:.4f}")

In [ ]:
print("\n" + "="*70)
print("FEATURE SELECTION - METHOD 4: F-Regression")
print("="*70)

# F-regression scores
selector_f = SelectKBest(f_regression, k=min(15, len(features_after_corr)))
selector_f.fit(X_train_scaled, y_train)

f_scores = selector_f.scores_
f_df = pd.DataFrame({
    'feature': features_after_corr,
    'f_score': f_scores
}).sort_values('f_score', ascending=False)

f_selected = [f for f, m in zip(features_after_corr, selector_f.get_support()) if m]

print(f"\nTop {len(f_selected)} features by F-Regression:")
for i, row in f_df.head(15).iterrows():
    print(f"  {row['feature']}: {row['f_score']:.2f}")

In [ ]:
print("\n" + "="*70)
print("FEATURE SELECTION - ENSEMBLE: Combine All Methods")
print("="*70)

# Count how many methods selected each feature
all_selected = set(rfe_selected) | set(sfm_selected) | set(mi_selected) | set(f_selected)

feature_votes = {}
for f in all_selected:
    votes = 0
    if f in rfe_selected: votes += 1
    if f in sfm_selected: votes += 1
    if f in mi_selected: votes += 1
    if f in f_selected: votes += 1
    feature_votes[f] = votes

# Sort by votes and correlation
feature_ranking = sorted(
    feature_votes.items(), 
    key=lambda x: (x[1], abs(target_corr.get(x[0], 0))), 
    reverse=True
)

print("\nFeature Ranking by Selection Method Agreement:")
print("-" * 60)
for i, (feat, votes) in enumerate(feature_ranking, 1):
    methods = []
    if feat in rfe_selected: methods.append('RFE')
    if feat in sfm_selected: methods.append('SFM')
    if feat in mi_selected: methods.append('MI')
    if feat in f_selected: methods.append('F')
    corr = target_corr.get(feat, 0)
    print(f"  {i:2d}. {feat}: {votes}/4 votes, corr={corr:.4f} ({', '.join(methods)})")

# Select features with >= 2 votes
final_features = [f for f, v in feature_ranking if v >= 2]

# Limit to top 12 for small dataset
if len(final_features) > 12:
    final_features = final_features[:12]

print(f"\n✓ Final feature set: {len(final_features)} features (>=2 votes)")

In [ ]:
print("\n" + "="*70)
print("FINAL FEATURE SET")
print("="*70)

print(f"\nFinal {len(final_features)} features selected:")
for i, feat in enumerate(final_features, 1):
    corr = target_corr.get(feat, 0)
    print(f"  {i:2d}. {feat} (corr: {corr:.4f})")

# Update train/test sets with final features
X_train_final = train_data[final_features]
X_test_final = test_data[final_features]

# Re-scale with final features only
scaler_final = StandardScaler()
X_train_scaled_final = scaler_final.fit_transform(X_train_final)
X_test_scaled_final = scaler_final.transform(X_test_final)

print(f"\n✓ Final scaled datasets ready")
print(f"   X_train: {X_train_scaled_final.shape}")
print(f"   X_test: {X_test_scaled_final.shape}")

---
## STEP 10: Model Training & Testing

**Models:** Ridge, Lasso, ElasticNet only

In [ ]:
print("="*70)
print("MODEL 1: RIDGE REGRESSION")
print("="*70)

# Use RidgeCV to find optimal alpha
alphas = np.logspace(-2, 6, 100)

ridge_cv = RidgeCV(
    alphas=alphas,
    cv=TimeSeriesSplit(n_splits=3),
    scoring='r2'
)
ridge_cv.fit(X_train_scaled_final, y_train)

print(f"\nOptimal alpha: {ridge_cv.alpha_:.4f}")

# Train R²
y_train_pred_ridge = ridge_cv.predict(X_train_scaled_final)
train_r2_ridge = r2_score(y_train, y_train_pred_ridge)
train_mape_ridge = mean_absolute_percentage_error(y_train, y_train_pred_ridge) * 100

# Test R²
y_test_pred_ridge = ridge_cv.predict(X_test_scaled_final)
test_r2_ridge = r2_score(y_test, y_test_pred_ridge)
test_mae_ridge = mean_absolute_error(y_test, y_test_pred_ridge)
test_mape_ridge = mean_absolute_percentage_error(y_test, y_test_pred_ridge) * 100

print(f"\n📊 Ridge Results:")
print(f"   Train R²: {train_r2_ridge:.4f}")
print(f"   Train MAPE: {train_mape_ridge:.2f}%")
print(f"   Test R²: {test_r2_ridge:.4f}")
print(f"   Test MAE: ${test_mae_ridge:,.0f}")
print(f"   Test MAPE: {test_mape_ridge:.2f}%")
print(f"   Overfit Gap: {train_r2_ridge - test_r2_ridge:.4f}")

In [ ]:
print("\n" + "="*70)
print("MODEL 2: LASSO REGRESSION")
print("="*70)

# Use LassoCV to find optimal alpha
lasso_cv = LassoCV(
    alphas=np.logspace(-2, 6, 100),
    cv=TimeSeriesSplit(n_splits=3),
    max_iter=10000,
    random_state=42
)
lasso_cv.fit(X_train_scaled_final, y_train)

print(f"\nOptimal alpha: {lasso_cv.alpha_:.4f}")

# Train R²
y_train_pred_lasso = lasso_cv.predict(X_train_scaled_final)
train_r2_lasso = r2_score(y_train, y_train_pred_lasso)
train_mape_lasso = mean_absolute_percentage_error(y_train, y_train_pred_lasso) * 100

# Test R²
y_test_pred_lasso = lasso_cv.predict(X_test_scaled_final)
test_r2_lasso = r2_score(y_test, y_test_pred_lasso)
test_mae_lasso = mean_absolute_error(y_test, y_test_pred_lasso)
test_mape_lasso = mean_absolute_percentage_error(y_test, y_test_pred_lasso) * 100

print(f"\n📊 Lasso Results:")
print(f"   Train R²: {train_r2_lasso:.4f}")
print(f"   Train MAPE: {train_mape_lasso:.2f}%")
print(f"   Test R²: {test_r2_lasso:.4f}")
print(f"   Test MAE: ${test_mae_lasso:,.0f}")
print(f"   Test MAPE: {test_mape_lasso:.2f}%")
print(f"   Overfit Gap: {train_r2_lasso - test_r2_lasso:.4f}")

# Show which features Lasso kept
lasso_coefs = pd.DataFrame({
    'feature': final_features,
    'coefficient': lasso_cv.coef_
})
lasso_nonzero = lasso_coefs[lasso_coefs['coefficient'] != 0].sort_values('coefficient', key=abs, ascending=False)
print(f"\nLasso kept {len(lasso_nonzero)} features (non-zero coefficients):")
for _, row in lasso_nonzero.iterrows():
    print(f"   {row['feature']}: {row['coefficient']:.2f}")

In [ ]:
print("\n" + "="*70)
print("MODEL 3: ELASTICNET REGRESSION")
print("="*70)

# Use ElasticNetCV to find optimal alpha and l1_ratio
l1_ratios = [0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99]

enet_cv = ElasticNetCV(
    l1_ratio=l1_ratios,
    alphas=np.logspace(-2, 6, 50),
    cv=TimeSeriesSplit(n_splits=3),
    max_iter=10000,
    random_state=42
)
enet_cv.fit(X_train_scaled_final, y_train)

print(f"\nOptimal alpha: {enet_cv.alpha_:.4f}")
print(f"Optimal l1_ratio: {enet_cv.l1_ratio_:.2f}")

# Train R²
y_train_pred_enet = enet_cv.predict(X_train_scaled_final)
train_r2_enet = r2_score(y_train, y_train_pred_enet)
train_mape_enet = mean_absolute_percentage_error(y_train, y_train_pred_enet) * 100

# Test R²
y_test_pred_enet = enet_cv.predict(X_test_scaled_final)
test_r2_enet = r2_score(y_test, y_test_pred_enet)
test_mae_enet = mean_absolute_error(y_test, y_test_pred_enet)
test_mape_enet = mean_absolute_percentage_error(y_test, y_test_pred_enet) * 100

print(f"\n📊 ElasticNet Results:")
print(f"   Train R²: {train_r2_enet:.4f}")
print(f"   Train MAPE: {train_mape_enet:.2f}%")
print(f"   Test R²: {test_r2_enet:.4f}")
print(f"   Test MAE: ${test_mae_enet:,.0f}")
print(f"   Test MAPE: {test_mape_enet:.2f}%")
print(f"   Overfit Gap: {train_r2_enet - test_r2_enet:.4f}")

---
## STEP 11: Advanced Hyperparameter Tuning

In [ ]:
print("="*70)
print("HYPERPARAMETER TUNING - RIDGE")
print("="*70)

# Fine-grained GridSearchCV for Ridge
best_alpha_ridge = ridge_cv.alpha_
alpha_range = np.linspace(max(0.01, best_alpha_ridge * 0.1), best_alpha_ridge * 10, 100)

ridge_grid = GridSearchCV(
    Ridge(),
    param_grid={'alpha': alpha_range},
    cv=TimeSeriesSplit(n_splits=5),
    scoring='r2',
    refit=True
)
ridge_grid.fit(X_train_scaled_final, y_train)

print(f"\nFine-tuned Ridge alpha: {ridge_grid.best_params_['alpha']:.4f}")
print(f"CV Score: {ridge_grid.best_score_:.4f}")

# Evaluate fine-tuned Ridge
y_train_pred_ridge_tuned = ridge_grid.predict(X_train_scaled_final)
y_test_pred_ridge_tuned = ridge_grid.predict(X_test_scaled_final)

train_r2_ridge_tuned = r2_score(y_train, y_train_pred_ridge_tuned)
test_r2_ridge_tuned = r2_score(y_test, y_test_pred_ridge_tuned)
test_mae_ridge_tuned = mean_absolute_error(y_test, y_test_pred_ridge_tuned)
test_mape_ridge_tuned = mean_absolute_percentage_error(y_test, y_test_pred_ridge_tuned) * 100

print(f"\n📊 Fine-Tuned Ridge Results:")
print(f"   Train R²: {train_r2_ridge_tuned:.4f}")
print(f"   Test R²: {test_r2_ridge_tuned:.4f}")
print(f"   Test MAE: ${test_mae_ridge_tuned:,.0f}")
print(f"   Test MAPE: {test_mape_ridge_tuned:.2f}%")

In [ ]:
print("\n" + "="*70)
print("HYPERPARAMETER TUNING - ELASTICNET")
print("="*70)

# Fine-grained GridSearchCV for ElasticNet
best_alpha_enet = enet_cv.alpha_
best_l1_enet = enet_cv.l1_ratio_

enet_grid = GridSearchCV(
    ElasticNet(max_iter=10000, random_state=42),
    param_grid={
        'alpha': np.linspace(max(0.01, best_alpha_enet * 0.5), best_alpha_enet * 2, 30),
        'l1_ratio': np.linspace(max(0.1, best_l1_enet - 0.2), min(0.99, best_l1_enet + 0.2), 15)
    },
    cv=TimeSeriesSplit(n_splits=5),
    scoring='r2',
    refit=True
)
enet_grid.fit(X_train_scaled_final, y_train)

print(f"\nFine-tuned ElasticNet alpha: {enet_grid.best_params_['alpha']:.4f}")
print(f"Fine-tuned ElasticNet l1_ratio: {enet_grid.best_params_['l1_ratio']:.4f}")
print(f"CV Score: {enet_grid.best_score_:.4f}")

# Evaluate fine-tuned ElasticNet
y_train_pred_enet_tuned = enet_grid.predict(X_train_scaled_final)
y_test_pred_enet_tuned = enet_grid.predict(X_test_scaled_final)

train_r2_enet_tuned = r2_score(y_train, y_train_pred_enet_tuned)
test_r2_enet_tuned = r2_score(y_test, y_test_pred_enet_tuned)
test_mae_enet_tuned = mean_absolute_error(y_test, y_test_pred_enet_tuned)
test_mape_enet_tuned = mean_absolute_percentage_error(y_test, y_test_pred_enet_tuned) * 100

print(f"\n📊 Fine-Tuned ElasticNet Results:")
print(f"   Train R²: {train_r2_enet_tuned:.4f}")
print(f"   Test R²: {test_r2_enet_tuned:.4f}")
print(f"   Test MAE: ${test_mae_enet_tuned:,.0f}")
print(f"   Test MAPE: {test_mape_enet_tuned:.2f}%")

---
## STEP 12: Model Comparison & Final Selection

In [ ]:
print("="*70)
print("MODEL COMPARISON")
print("="*70)

# Compile results
results = [
    ('Ridge (CV)', train_r2_ridge, test_r2_ridge, test_mape_ridge, test_mae_ridge),
    ('Ridge (Tuned)', train_r2_ridge_tuned, test_r2_ridge_tuned, test_mape_ridge_tuned, test_mae_ridge_tuned),
    ('Lasso (CV)', train_r2_lasso, test_r2_lasso, test_mape_lasso, test_mae_lasso),
    ('ElasticNet (CV)', train_r2_enet, test_r2_enet, test_mape_enet, test_mae_enet),
    ('ElasticNet (Tuned)', train_r2_enet_tuned, test_r2_enet_tuned, test_mape_enet_tuned, test_mae_enet_tuned)
]

results_df = pd.DataFrame(results, columns=['Model', 'Train R²', 'Test R²', 'Test MAPE (%)', 'Test MAE ($)'])
results_df['Overfit Gap'] = results_df['Train R²'] - results_df['Test R²']
results_df = results_df.sort_values('Test R²', ascending=False)

print("\n" + results_df.to_string(index=False))

# Best model
best_model_name = results_df.iloc[0]['Model']
best_test_r2 = results_df.iloc[0]['Test R²']
best_mape = results_df.iloc[0]['Test MAPE (%)']
best_mae = results_df.iloc[0]['Test MAE ($)']

print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"   Test R²: {best_test_r2:.4f}")
print(f"   Test MAPE: {best_mape:.2f}%")
print(f"   Test MAE: ${best_mae:,.0f}")

In [ ]:
# Visualization of Results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. R² Comparison
ax1 = axes[0]
x = np.arange(len(results_df))
width = 0.35

bars1 = ax1.bar(x - width/2, results_df['Train R²'], width, label='Train R²', color='steelblue', alpha=0.8)
bars2 = ax1.bar(x + width/2, results_df['Test R²'], width, label='Test R²', color='darkorange', alpha=0.8)

ax1.set_xlabel('Model')
ax1.set_ylabel('R² Score')
ax1.set_title('R² Comparison: Train vs Test')
ax1.set_xticks(x)
ax1.set_xticklabels(results_df['Model'], rotation=45, ha='right')
ax1.legend()
ax1.axhline(y=0, color='red', linestyle='--', alpha=0.5)
ax1.set_ylim(min(0, results_df['Test R²'].min() - 0.1), 1.0)

# 2. MAPE Comparison
ax2 = axes[1]
bars = ax2.bar(results_df['Model'], results_df['Test MAPE (%)'], color='seagreen', alpha=0.8)
ax2.set_xlabel('Model')
ax2.set_ylabel('Test MAPE (%)')
ax2.set_title('Test MAPE Comparison (Lower is Better)')
ax2.set_xticklabels(results_df['Model'], rotation=45, ha='right')

plt.tight_layout()
plt.show()

---
## STEP 13: Feature Importance Analysis

In [ ]:
print("="*70)
print("FEATURE IMPORTANCE (Best Model Coefficients)")
print("="*70)

# Get coefficients from best model
if 'Ridge' in best_model_name:
    if 'Tuned' in best_model_name:
        best_model = ridge_grid.best_estimator_
    else:
        best_model = ridge_cv
elif 'Lasso' in best_model_name:
    best_model = lasso_cv
else:
    if 'Tuned' in best_model_name:
        best_model = enet_grid.best_estimator_
    else:
        best_model = enet_cv

# Feature importance
importance_df = pd.DataFrame({
    'feature': final_features,
    'coefficient': best_model.coef_
})
importance_df['abs_coef'] = importance_df['coefficient'].abs()
importance_df = importance_df.sort_values('abs_coef', ascending=False)

print(f"\n{best_model_name} Feature Importance:")
print("-" * 50)
for i, row in importance_df.iterrows():
    sign = '+' if row['coefficient'] > 0 else '-'
    print(f"  {row['feature']}: {sign}{abs(row['coefficient']):,.2f}")

In [ ]:
# Visualize Feature Importance
fig, ax = plt.subplots(figsize=(10, 6))

colors = ['green' if c > 0 else 'red' for c in importance_df['coefficient']]
ax.barh(importance_df['feature'], importance_df['coefficient'], color=colors, alpha=0.7)
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
ax.set_xlabel('Coefficient Value')
ax.set_title(f'{best_model_name} - Feature Coefficients')
ax.invert_yaxis()

plt.tight_layout()
plt.show()

---
## STEP 14: Prediction Analysis

In [ ]:
print("="*70)
print("PREDICTION ANALYSIS")
print("="*70)

# Get predictions from best model
if 'Ridge' in best_model_name:
    if 'Tuned' in best_model_name:
        best_predictions = y_test_pred_ridge_tuned
    else:
        best_predictions = y_test_pred_ridge
elif 'Lasso' in best_model_name:
    best_predictions = y_test_pred_lasso
else:
    if 'Tuned' in best_model_name:
        best_predictions = y_test_pred_enet_tuned
    else:
        best_predictions = y_test_pred_enet

# Prediction comparison table
pred_df = pd.DataFrame({
    'Month': test_data['month'].values,
    'Year': test_data['year'].values,
    'Actual': y_test.values,
    'Predicted': best_predictions,
    'Error': y_test.values - best_predictions,
    'Error %': ((y_test.values - best_predictions) / y_test.values) * 100
})

print("\n2025 Predictions vs Actual:")
print("-" * 80)
for _, row in pred_df.iterrows():
    print(f"  {row['Month']} {int(row['Year'])}: " +
          f"Actual=${row['Actual']:,.0f}, " +
          f"Pred=${row['Predicted']:,.0f}, " +
          f"Error={row['Error %']:.2f}%")

In [ ]:
# Visualization: Actual vs Predicted
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. Time series comparison
ax1 = axes[0]
months = range(1, len(y_test) + 1)
ax1.plot(months, y_test.values, 'bo-', label='Actual', linewidth=2, markersize=8)
ax1.plot(months, best_predictions, 'r^--', label='Predicted', linewidth=2, markersize=8)
ax1.fill_between(months, y_test.values, best_predictions, alpha=0.2, color='gray')
ax1.set_xlabel('Month (2025)')
ax1.set_ylabel('Revenue ($)')
ax1.set_title('2025 Revenue: Actual vs Predicted')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Scatter plot
ax2 = axes[1]
ax2.scatter(y_test, best_predictions, c='steelblue', s=100, alpha=0.7, edgecolors='black')

# Perfect prediction line
min_val = min(y_test.min(), best_predictions.min())
max_val = max(y_test.max(), best_predictions.max())
ax2.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')

ax2.set_xlabel('Actual Revenue ($)')
ax2.set_ylabel('Predicted Revenue ($)')
ax2.set_title(f'Actual vs Predicted (R² = {best_test_r2:.4f})')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## STEP 15: Residual Analysis

In [ ]:
# Residual Analysis
residuals = y_test.values - best_predictions

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 1. Residual distribution
ax1 = axes[0]
ax1.hist(residuals, bins=10, color='steelblue', edgecolor='black', alpha=0.7)
ax1.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax1.set_xlabel('Residual ($)')
ax1.set_ylabel('Frequency')
ax1.set_title('Residual Distribution')

# 2. Residuals vs Predicted
ax2 = axes[1]
ax2.scatter(best_predictions, residuals, c='steelblue', s=80, alpha=0.7, edgecolors='black')
ax2.axhline(y=0, color='red', linestyle='--', linewidth=2)
ax2.set_xlabel('Predicted Revenue ($)')
ax2.set_ylabel('Residual ($)')
ax2.set_title('Residuals vs Predicted')

# 3. Q-Q Plot
ax3 = axes[2]
stats.probplot(residuals, dist="norm", plot=ax3)
ax3.set_title('Q-Q Plot (Normality Check)')

plt.tight_layout()
plt.show()

# Residual statistics
print(f"\nResidual Statistics:")
print(f"  Mean: ${np.mean(residuals):,.0f}")
print(f"  Std: ${np.std(residuals):,.0f}")
print(f"  Min: ${np.min(residuals):,.0f}")
print(f"  Max: ${np.max(residuals):,.0f}")

---
## STEP 16: Final Summary

In [ ]:
print("="*70)
print("FINAL SUMMARY")
print("="*70)

print(f"""
📊 PIPELINE RESULTS:

1. DATA:
   - Total samples: {len(df_monthly)}
   - Train (2023-2024): {len(train_data)} months
   - Test (2025): {len(test_data)} months

2. FEATURE ENGINEERING:
   - Total features created: {len(feature_cols)}
   - Features after variance filter: {len(features_after_var)}
   - Features after multicollinearity removal: {len(features_after_corr)}
   - Final features selected: {len(final_features)}

3. KEY DOMAIN FEATURES CREATED:
   ✓ weighted_revenue (expected value = forecast × prob)
   ✓ revenue_momentum_1m (lag1 - lag2, growth indicator)
   ✓ loading_factor (month-in-quarter loading)
   ✓ gap_ratio (weighted/unweighted, pipeline maturity)
   ✓ coverage_pipeline (pipeline sufficiency)
   ✓ maturity_rate (conversion efficiency)
   ✓ structural_feature (quarter loading structure)
   ✓ pipeline_stability (1/variance, consistency score)

4. FEATURE SELECTION METHODS USED:
   ✓ RFE (Recursive Feature Elimination)
   ✓ SelectFromModel (Lasso-based)
   ✓ Mutual Information
   ✓ F-Regression
   ✓ Ensemble voting (features with >=2 votes selected)

5. BEST MODEL: {best_model_name}
   - Test R²: {best_test_r2:.4f}
   - Test MAPE: {best_mape:.2f}%
   - Test MAE: ${best_mae:,.0f}

6. FINAL FEATURES ({len(final_features)}):""")

for i, feat in enumerate(final_features, 1):
    print(f"   {i:2d}. {feat}")

print(f"""
7. MODEL COMPARISON:
{results_df.to_string(index=False)}

8. PIPELINE FOLLOWED:
   ✓ Feature Engineering (19 phases)
   ✓ Leakage Checks (correlation + linear combination + business logic)
   ✓ Missing Value Handling (fillna + inf replacement)
   ✓ Proper Data Splitting (temporal train/test)
   ✓ Encoding & Scaling (StandardScaler)
   ✓ Correlation & Redundancy Removal (variance + multicollinearity)
   ✓ Feature Selection (RFE + SFM + MI + F-regression ensemble)
   ✓ Model Test (Ridge, Lasso, ElasticNet)
""")